In [23]:
# вариант с полной очисткой данных
from nltk.corpus import stopwords
import string
import re 
import os
import json


# очистка данных
months = ["Январ", "Феврал", "Март", "Апрел", "Ма", "Июн", "Июл", "Август", "Сентябр", "Октябр", "Ноябр", "Декабр"]
pattern = r'\b(?:' + '|'.join([month + '[а-я]*' for month in months]) + r')\b[.,]?' 
laws_links = []

def remove_stops(text, stops):

    text = re.sub(r'[а-яА-Я]\)\s', ' ', text) # удалить а), б) ...
    text = re.sub(r'\b[IVXLCDMivxlcdm]+\b', '', text) # удалить все римские цифры
    # text = re.sub(pattern, '', text, flags=re.IGNORECASE) # удалить месяцы
    
    #text = re.sub(r'\bг\.', '', text)
    #text = re.sub(r'\s\d{1,2}\.\s|\t\d{1,2}\.\s', ' ', text) # удалить 1. 2. ...
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'№\s', '№', text)# конкатенация №ФЗ123
    text = re.sub(r'(\t\d\s)|(\s\d\s)|(\s\d\t)', ' ', text)
    text = re.sub('•|№', '', text) # удаление мусора по типу '•'

    words = text.split()
    final = []
    final.append(' ')
    for word in words: # удалить стоп-слова
        if word not in stops:
            final.append(word)
    final = " ".join(final) # склеить список из строк
    final = final.translate(str.maketrans("", "", string.punctuation)) # удалить пунктуацию
    final = re.sub(r'(т\sд)|(\sт\sд)|(\sт\s)|((\sг\s))', '', final)
    
    #закинем в конец дока ссылки на законы в документе (по другому никак если оставлять только их)
    # z = text.split(' ')
    # for ind, i in enumerate(z):
    #     if '№' in i:
    #         laws_links.append(i)
            
    
    # final = "".join([i for i in final if not i.isdigit()]) # удалить все цифры
    final = re.sub(r'\t', ' ', final)
    final = re.sub(r'\s{1,}', ' ', final) # удалить много пробелов
    
    return (final)    


def clean_docs(docs):
    stops = stopwords.words('russian')
    final_text = []
    for i, doc in enumerate(docs):
        if i == 0 or i == 1:    # не трогать название документа
            final_text.append(doc)
        else:
            clean_doc = remove_stops(doc, stops)
            final_text.append(clean_doc)
            
    # if laws_links:
    #     final_text.append('\n')         
    #     final_text.append(' '.join(laws_links))
    
    # laws_links.clear
        
    return (final_text)

In [6]:
# вариант очистки данных максимально приближенный к оригиналу 
import re 
import os

# очистка данных

def cleanser(text):

    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'№\s', '№', text)# конкатенация №ФЗ123
    text = re.sub(r'(\t\d\s)|(\s\d\s)|(\s\d\t)', ' ', text)
    text = re.sub('•|№', '', text) # удаление мусора по типу '•'    
    
    return (text)    


def clean_docs(docs):
    
    final_text = []
    for i, doc in enumerate(docs):
        if i == 0 or i == 1:    # не трогать название документа
            final_text.append(doc)
        else:
            clean_doc = cleanser(doc)
            final_text.append(clean_doc)

    return (final_text)

In [18]:
file = open('/Users/gleb/Desktop/InternetLab/internetlab/docs/row_docs/Стратегия_развития_информационного_общества_в_Российской_Федерации_на_2017_-_2030_годы.txt', 'r', encoding='utf-8')

cleaned_docs = clean_docs(file)

# запись в файл
with open('/Users/gleb/Desktop/InternetLab/internetlab/docs//clean_docs/c_Стратегия_развития_информационного_общества_в_Российской_Федерации_на_2017_-_2030_годы.txt', "w", encoding="utf-8") as d:
    for i in cleaned_docs:
        d.write(i)
    

In [7]:
# загрузка приближенных к оригиналу доков

r_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/row_docs/'
c_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs//clean_docs/'

raw_doc_list = os.listdir(r_path)
clean_doc_list = os.listdir(c_path)
for doc in raw_doc_list:
    if '.txt' in doc:
        file  = open(os.path.join(r_path, doc), 'r', encoding='utf-8')
        cleaned_docs = clean_docs(file)
        # запись в файл
        for c_doc in clean_doc_list:
            document = os.path.join(c_path, 'c_' + doc)
            with open(document, "w", encoding="utf-8") as d:
                for i in cleaned_docs:
                    d.write(i)
            break
            

In [22]:
# загрузка  полностью очищеных доков

r_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/row_docs/raw_fed'
c_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/fed_clean/fed_full_clean'

raw_doc_list = os.listdir(r_path)
clean_doc_list = os.listdir(c_path)
for doc in raw_doc_list:
    if '.txt' in doc:
        file  = open(os.path.join(r_path, doc), 'r', encoding='utf-8')
        cleaned_docs = clean_docs(file)
        # запись в файл
    
        document = os.path.join(c_path, 'fc_' + doc)
        with open(document, "w", encoding="utf-8") as d:
            for i in cleaned_docs:
                d.write(i)

In [27]:
# Запись в Json
json_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/clean_fed.json'
folder_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/fed_clean/fed_full_clean'

# Создаем пустые списки для названий документов и текста
titles = []
texts = []

# Перебираем файлы в папке 'fed_full_clean'
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            lines = file.readlines()
            if len(lines) >= 3:  # Убеждаемся, что в файле достаточно строк
                titles.append(lines[0].strip())  # Добавляем название документа из первой строки
                texts.append(lines[2].strip())  # Добавляем текст из третьей строки

# Создаем словарь с названиями документов и текстом
data = {
    "title": titles,
    "text": texts
}

# Записываем данные в JSON файл
with open(json_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)